# Data Cleaning and Preparation | Limpeza de Dados e Preparação

## Objective | Objetivo

- Load raw data from Lipstick Casino | Carregar os dados brutos do Lipstick Casino
- Perform necessary cleaning transformations | Realziar limpeza e transformações necessárias
- Save processed data for later analysis | Salvar dados processados para análise posterior


## 1. Loading and Initial Inspection | Carregamento e Inspeção Inicial

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure Paths | Configurar Paths
current_dir = Path.cwd()
data_dir = current_dir.parent / 'data'
raw_data_path = data_dir / 'Lipstick_casino_data.xlsx'
processed_data_path = data_dir / 'processed' / 'casino_data_processed.parquet'

# Create directory for processed data | Criar diretório para dados processados
(data_dir / 'processed').mkdir(parents=True, exist_ok=True)

In [ ]:
# Load raw data | Carregar dados brutos
print("Loading raw data...")
raw_data = pd.read_excel(raw_data_path, sheet_name='Casino Data')

# Show basic infos | Mostrar informações básicas
print("\nInfos abouth raw data:")
print(f"Total records: {len(raw_data)}")
print(f"Total columns: {len(raw_data.columns)}")
print("\nFirst 5 rows:")
display(raw_data.head())

In [ ]:
# Checking missing values | Verificar valores ausentes
print("\nMissing values per column:")
print(raw_data.isnull().sum())

In [ ]:
# Data preprocessing | Pré-processamento dos dados
print("\nStarting preprocessing...")

# Convert date column| Converter coluna de data
raw_data['Month, Year'] = pd.to_datetime(raw_data['Month, Year'])

# Create new temporal features | Criar novas features temporais
raw_data['Year'] = raw_data['Month, Year'].dt.year
raw_data['Quarter'] = raw_data['Month, Year'].dt.quarter
raw_data['Month_Name'] = raw_data['Month, Year'].dt.month_name()

# Normalize categories | Normalizar categorias
raw_data['Game category'] = raw_data['Game category'].str.title().str.strip()
raw_data['Country'] = raw_data['Country'].str.title().str.strip()
raw_data['Table Type Commercial'] = raw_data['Table Type Commercial'].str.strip()

# Calculate Hold % (GGR / Bet Volume) | Calcular Hold % (GGR / Volume de Apostas)
raw_data['Hold_Pct'] = raw_data['GGR Ucur'] / raw_data['Wager Ucur']
raw_data['Hold_Pct'] = raw_data['Hold_Pct'].replace([np.inf, -np.inf], np.nan)

# Handling extreme negative values | Tratar valores negativos extremos
raw_data.loc[raw_data['GGR Ucur'] < -10000, 'GGR Ucur'] = np.nan

# Consolidate table types | Consolidar tipos de mesa
raw_data['Table_Type_Simplified'] = np.where(
    raw_data['Table Type Commercial'].str.contains('High Stakes', case=False),
    'High Stakes',
    'Regular'
)

In [ ]:
# Save processed data |  Salvar dados processados
raw_data.to_parquet(processed_data_path)
print(f"\nProcessed data saved in: {processed_data_path}")
print("Pre-processing completed successfully!")